In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import numpy
import math
import string
import re
from collections import Counter

In [2]:
import konlpy

In [3]:
# 관용어구 
# 사전에 manually 단어 추가 (ex. 렌탈) --> inkoo.txt
from konlpy.tag import Komoran
#from konlpy.tag import mecab
text = "오늘 서울의 날씨는 추워질 전망입니다. 오늘 오후 한때 소나기가 올 예정입니다."
tagger = Komoran(userdic = 'inkoo.txt')
tags = tagger.pos(text)
tags

[('오늘', 'NNG'),
 ('서울', 'NNP'),
 ('의', 'JKG'),
 ('날씨', 'NNG'),
 ('는', 'JX'),
 ('춥', 'VA'),
 ('어', 'EC'),
 ('지', 'VX'),
 ('ㄹ', 'ETM'),
 ('전망', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF'),
 ('오늘', 'NNG'),
 ('오후', 'NNG'),
 ('한때', 'NNG'),
 ('소나기', 'NNP'),
 ('가', 'JKS'),
 ('올', 'MM'),
 ('예정', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF')]

In [4]:
df = pd.read_excel('DART-2021_extracted.xlsx')
df

,회사명,사업 내용 (extracted)
0,3S,사업의 내용1. 사업의개요당사는 반도체 웨이퍼 캐리어 사업과 환경시험장치 사업을 주...
1,AJ네트웍스,사업의 내용조직도1. 사업의 개요가. 업계의 현황소유를 중시하던 전통적인 자산 개념...
2,AK홀딩스,사업의 내용1. 사업의 개요지주사업 부문(1) 업계의 현황1) 산업의 특성 등① 산...
3,APS홀딩스,사업의 내용1. 사업의 개요가. 업계의 현황지주회사란 다른 회사의 주식소유를 목적으...
4,AP시스템,사업의 내용1. 사업의 개요가. 산업별 현황 [OLED 장비산업] 1) 산업의 특성...
...,...,...
2827,흥국화재,사업의 내용1. 사업의 개요가. 업계의 현황(1) 산업의 특성 및 성장성- 손해보험...
2828,흥아해운,사업의 내용1. 사업의 개요가. 업계의 현황(1) 해운업의 특성해운업은 선박의 공간...
2829,희림,사업의 내용1.사업의 개요가. 산업의 특성당사는 건설 관련 서비스업인 건축물의 설계...
2830,힐세리온,사업의 내용1. 사업의 개요 주식회사 힐세리온은 핸드핼드 초음파 진단기(Sonon)...


In [5]:
# 3S라는 기업의 사업 내용에 사용된 단어들 (중복 허용)
noun_list = []
tagged = tagger.pos(df.loc[0, '사업 내용 (extracted)'])
    
for word, tag in tagged:
    if tag in ['NNP', 'NNG']:
        noun_list.append(word)
            
noun_list = [re.sub('[^가-힣]', '', x) for x in noun_list]
noun_list = [x for x in noun_list if x]
noun_list

['사업',
 '내용',
 '사업',
 '개요',
 '당사',
 '반도체',
 '웨이퍼',
 '캐리어',
 '사업',
 '환경',
 '시험',
 '장치',
 '사업',
 '사업',
 '영위',
 '당사',
 '종속',
 '회사',
 '환경',
 '시험',
 '장치',
 '사업',
 '수행',
 '반도체',
 '웨이퍼',
 '캐리어',
 '사업',
 '산업',
 '특성',
 '반도체',
 '시스템',
 '산업',
 '발전',
 '원동력',
 '역할',
 '담당',
 '인공지능',
 '로봇',
 '기술',
 '생명과학',
 '산업',
 '혁명',
 '선도',
 '핵심',
 '중추',
 '산업',
 '발전',
 '최근',
 '인공지능',
 '자율',
 '주행',
 '자동차',
 '비전',
 '구체',
 '반도체',
 '시장',
 '속도',
 '당사',
 '영위',
 '반도체',
 '웨이퍼',
 '캐리어',
 '사업',
 '반도체',
 '주재료',
 '인실리코',
 '웨이퍼',
 '운송',
 '때',
 '사용',
 '웨이퍼',
 '이송',
 '박스',
 '생산',
 '사업',
 '실리콘',
 '웨이퍼',
 '반도체',
 '근간',
 '소재',
 '전자',
 '기기',
 '핵심',
 '부품',
 '대부분',
 '반도체',
 '기기',
 '칩',
 '제작',
 '기판',
 '재료',
 '사용',
 '역할',
 '담당',
 '취급',
 '상당',
 '주의',
 '필요',
 '실리콘',
 '웨이퍼',
 '시장',
 '시장',
 '참여자',
 '제한',
 '비교',
 '안정',
 '환경',
 '속',
 '성장',
 '반도체',
 '웨이퍼',
 '캐리어',
 '실리콘',
 '웨이퍼',
 '출하',
 '영향',
 '품목',
 '특성',
 '반도체',
 '웨이퍼',
 '캐리어',
 '사용',
 '용도',
 '종류',
 '웨이퍼',
 '생산',
 '업체',
 '공정',
 '사용',
 '박스',
 '웨이퍼',
 '생산',
 '업체',
 '반도체',
 '생산',
 '업체',
 '동시',

In [6]:
# 많이 사용된 순서로 나열
noun_counts = Counter(noun_list)
noun_counts.most_common()

[('웨이퍼', 52),
 ('생산', 41),
 ('반도체', 31),
 ('당사', 29),
 ('환경', 29),
 ('사업', 26),
 ('시장', 26),
 ('사용', 24),
 ('시험', 22),
 ('업체', 20),
 ('제품', 19),
 ('산업', 17),
 ('칼로리', 16),
 ('메타', 16),
 ('장치', 15),
 ('설비', 15),
 ('성장', 13),
 ('캐리어', 12),
 ('공정', 12),
 ('증가', 12),
 ('국내', 11),
 ('원재료', 11),
 ('지속', 11),
 ('공조', 11),
 ('시스템', 10),
 ('기술', 10),
 ('실리콘', 10),
 ('세계', 10),
 ('냉동', 10),
 ('출하', 9),
 ('자동차', 8),
 ('특수', 8),
 ('조사', 8),
 ('개발', 8),
 ('관련', 8),
 ('부문', 8),
 ('영향', 7),
 ('기술력', 7),
 ('제조', 7),
 ('국내외', 7),
 ('판매', 7),
 ('에어컨', 7),
 ('특성', 6),
 ('최근', 6),
 ('기기', 6),
 ('품목', 6),
 ('일본', 6),
 ('수요', 6),
 ('예상', 6),
 ('영업', 6),
 ('가격', 6),
 ('친환경', 6),
 ('확대', 6),
 ('매입', 6),
 ('박스', 5),
 ('필요', 5),
 ('용기', 5),
 ('수량', 5),
 ('변동', 5),
 ('공급', 5),
 ('강화', 5),
 ('장비', 5),
 ('각종', 5),
 ('뉴딜', 5),
 ('역할', 4),
 ('구체', 4),
 ('운송', 4),
 ('이송', 4),
 ('부품', 4),
 ('제작', 4),
 ('재료', 4),
 ('용도', 4),
 ('투자', 4),
 ('미국', 4),
 ('납품', 4),
 ('인해', 4),
 ('성장률', 4),
 ('기준', 4),
 ('예측', 4),
 ('현황', 4)

In [7]:
# 빈 데이터 프레임 생성
df2 = df.copy()
df2['사업 내용 (extracted)'] = ''
df2.rename(columns = {'사업 내용 (extracted)':'사업 내용 bow'}, inplace = True)
df2

,회사명,사업 내용 bow
0,3S,
1,AJ네트웍스,
2,AK홀딩스,
3,APS홀딩스,
4,AP시스템,
...,...,...
2827,흥국화재,
2828,흥아해운,
2829,희림,
2830,힐세리온,


In [19]:
# 모든 회사에 대해 많이 사용된 순서대로 (단어, 사용 횟수) 형태로 저장
for i in range(2830,2832):
    noun_list = []
    tagged = tagger.pos(df.loc[i, '사업 내용 (extracted)'])
    
    for word, tag in tagged:
        if tag in ['NNP', 'NNG']:
            noun_list.append(word)
    noun_list = [re.sub('[^가-힣]', '', x) for x in noun_list]
    noun_list = [x for x in noun_list if x]
    noun_counts = Counter(noun_list)
    result = noun_counts.most_common()
    df2.loc[i, '사업 내용 bow'] = result

In [20]:
df2

,회사명,사업 내용 bow
0,3S,"[(웨이퍼, 52), (생산, 41), (반도체, 31), (당사, 29), (환경..."
1,AJ네트웍스,"[(렌탈, 129), (시장, 86), (사업, 47), (장비, 41), (파렛트..."
2,AK홀딩스,"[(산업, 132), (시장, 114), (사업, 88), (제품, 78), (회사..."
3,APS홀딩스,"[(사업부, 20), (지주회사, 13), (회사, 11), (사업, 10), (부..."
4,AP시스템,"[(장비, 84), (산업, 55), (반도체, 48), (시장, 27), (업체,..."
...,...,...
2828,흥국화재,"[(보험, 78), (보험료, 35), (손해보험, 32), (회사, 29), (계..."
2829,흥아해운,"[(해운, 36), (사업, 18), (업, 18), (선박, 17), (세계, 1..."
2830,희림,"[(사업, 13), (시장, 12), (공사, 12), (설계, 11), (건설, ..."
2831,힐세리온,"[(초음파, 167), (진단, 130), (의료, 82), (개발, 53), (영..."


In [21]:
df2.to_excel("2021_bow_df.xlsx")

In [22]:
df2.to_pickle("2021_bow_df_pickle")

In [8]:
df2 = pd.read_pickle("2021_bow_df_pickle")
df2

,회사명,사업 내용 bow
0,3S,"[(웨이퍼, 52), (생산, 41), (반도체, 31), (당사, 29), (환경..."
1,AJ네트웍스,"[(렌탈, 129), (시장, 86), (사업, 47), (장비, 41), (파렛트..."
2,AK홀딩스,"[(산업, 132), (시장, 114), (사업, 88), (제품, 78), (회사..."
3,APS홀딩스,"[(사업부, 20), (지주회사, 13), (회사, 11), (사업, 10), (부..."
4,AP시스템,"[(장비, 84), (산업, 55), (반도체, 48), (시장, 27), (업체,..."
...,...,...
2828,흥국화재,"[(보험, 78), (보험료, 35), (손해보험, 32), (회사, 29), (계..."
2829,흥아해운,"[(해운, 36), (사업, 18), (업, 18), (선박, 17), (세계, 1..."
2830,희림,"[(사업, 13), (시장, 12), (공사, 12), (설계, 11), (건설, ..."
2831,힐세리온,"[(초음파, 167), (진단, 130), (의료, 82), (개발, 53), (영..."


In [9]:
# 전체 기업의 사업 내용 설명에 사용된 단어의 합집합 (noun list)
bow = []
for i in range(len(df2)):
    words = [word for word, count in df2.loc[i, '사업 내용 bow']]
    bow = bow + words

In [10]:
bow = list(set(bow))   # 중복 제거
len(bow)

34633

# Noun list에서 지명, common words 제거

In [8]:
# 지명
region = ['서울', '서울시', '경기도', '강원도', '충청도', '충청북도', '충청남도', '전라도', '전라북도', '전라남도', 
          '경상도', '경상북도', '경상남도', '제주도', '부산', '인천', '대구', '대전', '광주', '울산']

In [9]:
bow = [x for x in bow if x not in region]
len(bow)

34613

In [10]:
df3 = df2.copy()
df3['사업 내용 bow'] = ''
df3

,회사명,사업 내용 bow
0,3S,
1,AJ네트웍스,
2,AK홀딩스,
3,APS홀딩스,
4,AP시스템,
...,...,...
2828,흥국화재,
2829,흥아해운,
2830,희림,
2831,힐세리온,


In [11]:
for i in range(len(df2)):
    words = [word for word, count in df2.loc[i, '사업 내용 bow']]
    df3.loc[i, '사업 내용 bow'] = words
df3

,회사명,사업 내용 bow
0,3S,"[웨이퍼, 생산, 반도체, 당사, 환경, 사업, 시장, 사용, 시험, 업체, 제품,..."
1,AJ네트웍스,"[렌탈, 시장, 사업, 장비, 파렛트, 당사, 업체, 자산, 확대, 아이템, 기기,..."
2,AK홀딩스,"[산업, 시장, 사업, 제품, 회사, 부문, 년, 화장품, 항공, 특성, 성장, 국..."
3,APS홀딩스,"[사업부, 지주회사, 회사, 사업, 부문, 자산, 영업, 연결, 부동산, 주, 소프..."
4,AP시스템,"[장비, 산업, 반도체, 시장, 업체, 패널, 기술, 공정, 제조, 개발, 제품, ..."
...,...,...
2828,흥국화재,"[보험, 보험료, 손해보험, 회사, 계, 기준, 기제, 합, 계약, 영업, 단위, ..."
2829,흥아해운,"[해운, 사업, 업, 선박, 세계, 산업, 경제, 부문, 변동, 연간, 회사, 특성..."
2830,희림,"[사업, 시장, 공사, 설계, 건설, 국내, 해외, 기제, 감리, 당사, 투자, 건..."
2831,힐세리온,"[초음파, 진단, 의료, 개발, 영상, 혈류, 기기, 연구, 사용, 패치, 진료, ..."


In [12]:
df3.to_excel("2021_bow_df_only_words.xlsx")

In [13]:
df3.to_pickle("2021_bow_df_only_words_pickle")

In [11]:
df3 = pd.read_pickle("2021_bow_df_only_words_pickle")

In [15]:
# check
count = 0
for j in range(len(df3)):
    if '힐스' in df3.loc[j, '사업 내용 bow']:
        count += 1
print(count)

6


In [16]:
# 전체 기업 중 25% 이상의 기업이 사용한 단어를 추출
frequent = []
for i in bow:
    count = 0
    for j in range(len(df3)):
        if i in df3.loc[j, '사업 내용 bow']:
            count += 1
    if count > (len(df3) / 4):
        frequent.append(i)

In [17]:
print(len(frequent))
frequent

565


['규모',
 '비율',
 '기회',
 '경향',
 '전력',
 '물질',
 '추이',
 '용도',
 '침체',
 '주도',
 '상태',
 '평균',
 '전방',
 '단계',
 '사업',
 '계획',
 '상당',
 '종류',
 '비교',
 '시간',
 '부문',
 '확대',
 '확장',
 '투자',
 '최초',
 '개선',
 '자료',
 '하',
 '전',
 '경우',
 '예상',
 '제조',
 '구매',
 '배',
 '목적',
 '혁신',
 '소재',
 '영업',
 '자회사',
 '대기업',
 '필수',
 '업무',
 '채널',
 '공장',
 '류',
 '기업',
 '차지',
 '면',
 '시대',
 '향후',
 '완료',
 '담당',
 '연평균',
 '모델',
 '참여',
 '전자',
 '개념',
 '설립',
 '중소',
 '적용',
 '상',
 '절감',
 '계절',
 '업체',
 '합',
 '상대',
 '둔화',
 '역량',
 '법',
 '위주',
 '동향',
 '포함',
 '년대',
 '년월',
 '자체',
 '기',
 '경쟁사',
 '미',
 '확인',
 '반면',
 '당기',
 '응용',
 '차세대',
 '확산',
 '전반',
 '마케팅',
 '문화',
 '차별',
 '제품개발',
 '국내',
 '이용',
 '인터넷',
 '전기',
 '코로나',
 '제고',
 '노하우',
 '경쟁력',
 '현황',
 '연결',
 '산출',
 '첨단',
 '의미',
 '위험',
 '수준',
 '센터',
 '이전',
 '진출',
 '기능',
 '등장',
 '시설',
 '인수',
 '계속',
 '성과',
 '한국',
 '가격',
 '가운데',
 '유지',
 '스마트폰',
 '성장세',
 '방법',
 '판단',
 '우리나라',
 '연구',
 '계',
 '기술력',
 '운영',
 '소요',
 '일',
 '인해',
 '보호',
 '능력',
 '국제',
 '고도',
 '복합',
 '변경',
 '연간',
 '지배',
 '축적',
 '문제',
 '스마트',
 '계약',
 '납품'

In [83]:
# frequent 단어들을 txt로 저장
with open("2021_frequent_25%.txt", "w") as f:
    for s in frequent:
        f.write(s +"\n")

In [48]:
# frequent를 제외한 단어의 개수
bow = [x for x in bow if x not in frequent]
len(bow)

34048

In [50]:
bow = sorted(bow)

In [51]:
# frequent를 제거한 bow를 txt로 저장
with open("2021_bow.txt", "w") as f:
    for s in bow:
        f.write(s +"\n")

In [61]:
# manual check (아직 남아있는 이상한 단어 제거 (+ frequent에 들어 있지만 bow에 넣어줘야 하는 단어 다시 넣기))

In [12]:
# frequent를 제거하고 manual check을 거친 bow를 list로 불러오기
bow = []
with open("2021_bow.txt", "r") as f:
    for line in f:
        bow.append(str(line.strip()))
print(len(bow))
bow

34038


['가감',
 '가게',
 '가격결정',
 '가격상한',
 '가격연동제',
 '가격차별',
 '가격파괴',
 '가격표',
 '가결',
 '가경',
 '가경제',
 '가계',
 '가계도',
 '가계부',
 '가계부채',
 '가고',
 '가고시마',
 '가공량',
 '가공력',
 '가공료',
 '가공식품',
 '가공업자',
 '가공품',
 '가교',
 '가구',
 '가구원',
 '가군',
 '가규',
 '가금',
 '가까이',
 '가나',
 '가남읍',
 '가내',
 '가내수공업',
 '가노',
 '가늠',
 '가늠자',
 '가능',
 '가능동',
 '가니',
 '가니쉬',
 '가다랑어',
 '가닥',
 '가담',
 '가대',
 '가덕도',
 '가도',
 '가독',
 '가독성',
 '가동',
 '가두',
 '가드',
 '가드너',
 '가드레일',
 '가득',
 '가든',
 '가든시티',
 '가든파이브',
 '가디언',
 '가디언즈',
 '가라오케',
 '가락',
 '가락동',
 '가락시장',
 '가람',
 '가랑비',
 '가랑이',
 '가래',
 '가려움',
 '가려움증',
 '가로',
 '가로등',
 '가로변',
 '가로세로',
 '가로수',
 '가로수길',
 '가로수로',
 '가루',
 '가르가넬리',
 '가름',
 '가리',
 '가리개',
 '가리비',
 '가리온',
 '가마',
 '가마솥',
 '가망',
 '가맹',
 '가맹점',
 '가면',
 '가명',
 '가모',
 '가몬',
 '가문',
 '가문의영광',
 '가물',
 '가뭄',
 '가미',
 '가바',
 '가발',
 '가방',
 '가변',
 '가변밸브타이밍',
 '가변수',
 '가변저항',
 '가보',
 '가보세',
 '가봉',
 '가부',
 '가비',
 '가비아',
 '가사',
 '가사지원로봇',
 '가산',
 '가산금',
 '가산동',
 '가산면',
 '가상',
 '가상공간',
 '가상머신',
 '가상사설망',
 '가상세계',
 '가상키보드',
 '가상학습',
 '가상현실',
 '가상화',
 '가

In [78]:
# df2에서 bow에 있는 단어만 남기기
df5 = df2.copy()
for i in range(len(df5)):
    df5.loc[i, '사업 내용 bow'] = [x for x in df2.loc[i, '사업 내용 bow'] if x[0] in bow]
df5

,회사명,사업 내용 bow
0,3S,"[(웨이퍼, 52), (반도체, 31), (시험, 22), (칼로리, 16), (메..."
1,AJ네트웍스,"[(렌탈, 129), (파렛트, 35), (아이템, 21), (고소, 17), (물..."
2,AK홀딩스,"[(화장품, 42), (항공, 42), (석유화학, 35), (애경, 29), (용..."
3,APS홀딩스,"[(지주회사, 13), (부동산, 7), (소프트웨어, 6), (정밀, 6), (주..."
4,AP시스템,"[(반도체, 48), (패널, 20), (양산, 9), (디스플레이, 7), (삼성..."
...,...,...
2828,흥국화재,"[(보험, 78), (보험료, 35), (손해보험, 32), (운용, 17), (신..."
2829,흥아해운,"[(해운, 36), (선박, 17), (운송, 9), (선사, 7), (과세, 7)..."
2830,희림,"[(공사, 12), (감리, 7), (건축, 6), (도급액, 6), (서비스업, ..."
2831,힐세리온,"[(초음파, 167), (진단, 130), (영상, 48), (혈류, 48), (패..."


In [79]:
df5.to_excel("2021_bow_df_exclude_frequent.xlsx")

In [80]:
df5.to_pickle("2021_bow_df_exclude_frequent")

In [13]:
df5 = pd.read_pickle("2021_bow_df_exclude_frequent")

# etc.

In [ ]:
conda install -c conda-forge jpype1

In [ ]:
pip install konlpy

In [ ]:
result = [''.join([i for i in x if not i.isdigit()]) for x in s]
result = [(''.join(i for i in x if i not in string.punctuation)) for x in result]
result = [re.sub(r'[^\w]', ' ', x) for x in result]
result = [x for x in result if x]

In [ ]:
result = filter(str.strip, result)

In [4]:
from konlpy.tag import Kkma  # 꼬꼬마 형태소 분석기 사용
kkma = Kkma()
sentences = kkma.sentences(text)
sentences[1]

'오늘 오후 한때 소나기가 올 예정입니다.'

In [5]:
kkma.nouns(text)

['오늘', '서울', '날씨', '전망', '오후', '한때', '소나기', '예정', '렌', '장비']